In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import numpy as np
import nltk
import re
import string
from sklearn.model_selection import StratifiedShuffleSplit
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from collections import Counter
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, naive_bayes
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MaxAbsScaler
from scipy import stats
from sklearn.metrics import make_scorer, roc_auc_score
from scipy.linalg import svd
from numpy import diag
from scipy.sparse import csr_matrix
from gensim.models import Word2Vec, Doc2Vec, FastText
from gensim.models.doc2vec import TaggedDocument
from gensim.models.phrases import Phraser, Phrases
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from sklearn import svm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D,Bidirectional
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from spellchecker import SpellChecker

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

ModuleNotFoundError: ignored

In [ ]:
!pip install pyspellchecker
!pip install keras
!pip install nltk

In [ ]:
stop=set(stopwords.words('english'))

In [ ]:
np.random.seed(7712)
spell = SpellChecker()

In [ ]:
train_data = pd.read_csv("drive/My Drive/data/nlp-getting-started/train.csv")
train_data.head()
print(train_data.dtypes)

In [ ]:
test_data = pd.read_csv("drive/My Drive/data/nlp-getting-started/test.csv")
test_data.head()
print(train_data.dtypes)

In [ ]:
train_data = train_data.drop(columns=['id','keyword','location'])
test_data = test_data.drop(columns=['id','keyword','location'])
train_data.head()

In [ ]:
test_data['text']=test_data['text'].replace("","empty")
print(train_data.size)
print(test_data.size)

In [ ]:
df = pd.concat([train_data,test_data])
df.dtypes

In [ ]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',str(text))

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',str(text))

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

In [ ]:
df['text'] = df['text'].apply(lambda a : remove_URL(a))
df['text'] = df['text'].apply(lambda a : remove_html(a))
df['text'] = df['text'].apply(lambda a : remove_emoji(a))
df['text'] = df['text'].apply(lambda a : remove_punct(a))

In [ ]:
train_data = df[:train_data.shape[0]]
test_data = df[train_data.shape[0]:]
train_data.shape

In [ ]:
y=1
def correct_spellings(text):
    global y
    print(y)
    y = y+1
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if ("@" in word) or ("#" in word) or ("http" in word):
            corrected_text.append(word)
        elif word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

In [ ]:
train_data['text']=train_data['text'].apply(lambda x : correct_spellings(x))

In [ ]:
train_data.to_csv(r'drive/My Drive/data/nlp-getting-started/spellcheck_output_train2.csv')

In [ ]:
test_data['text']=test_data['text'].apply(lambda x : correct_spellings(x))
# test_data=pd.read_csv("spellcheck_output_test.csv")
# test_data = test_data.drop(['Unnamed: 0'], axis=1)
# test_data.head()
# test_data['text'] = test_data['text'].apply(lambda x: correct_spellings(x))

In [ ]:
test_data.to_csv(r'drive/My Drive/data/nlp-getting-started/spellcheck_output_test2.csv')

In [ ]:
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None

In [ ]:
count = train_data['target'].value_counts()
count

In [ ]:
def plot_bar(df, feat_x, feat_y, normalize=True):
    """ Plot with vertical bars of the requested dataframe and features"""
    
    ct = pd.crosstab(df[feat_x], df[feat_y])
    if normalize == True:
        ct = ct.div(ct.sum(axis=1), axis=0)
    return ct.plot(kind='bar', stacked=True)

In [ ]:
pd.value_counts(train_data['target']).plot.bar()

In [ ]:
splits = StratifiedShuffleSplit(n_splits=1,test_size=0.3,random_state=69)
for train_i, test_i in splits.split(train_data,train_data['target']):
    train_set = train_data.loc[train_i]
    test_set = train_data.loc[test_i]

In [ ]:
# train_set=train_set.reset_index(drop=True)
test_set.head()

In [ ]:
# train_set = process(train_set)
# test_set = process(test_set)

frames = [train_set,test_set]
train_data = pd.concat(frames)

train_data = train_data.reset_index(drop=True)
# test_data = process(test_data)

In [ ]:
train_set.head()

In [ ]:
def create_corpus(train_data):
    corpus=[]
    for tweet in train_data['text']:
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

In [ ]:
corpus=create_corpus(train_data)

In [ ]:
embedding_dict={}
with open('glove.6B.100d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

In [ ]:
MAX_LEN=50
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

text_padding=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [ ]:
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

In [ ]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))

for word,i in word_index.items():
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec

In [ ]:
model=Sequential()
n_timesteps = 10
embedding=Embedding(num_words,100,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)

model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2), input_shape=(n_timesteps, 1)))
# model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


optimzer=Adam(learning_rate=1e-5)

model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
train=text_padding[:train_data.shape[0]]
test=text_padding[train_data.shape[0]:]

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(train,train_data['target'].values,test_size=0.15)

In [ ]:
history=model.fit(X_train,y_train,batch_size=4,epochs=15,validation_data=(X_test,y_test),verbose=2)

In [ ]:
test1=text_padding[:test_data.shape[0]]
test2=text_padding[test_data.shape[0]:]
predictions=history.model.predict_classes(x=test1,batch_size=None,verbose=1)

In [ ]:
output_df = pd.DataFrame(predictions,columns=['target'])
output_df

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators=100, random_state = 1)
model.fit(X_train, y_train)
test_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, test_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
Rando= RandomForestClassifier(n_estimators=100)
Rando.fit(X_train,y_train)
y_pred1=Rando.predict(X_test)
accuracy_score(y_test,y_pred1)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(C=1e5)
logreg.fit(X_train,y_train)
y_pred2=logreg.predict(X_test)
accuracy_score(y_test,y_pred2)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
svc = DecisionTreeClassifier().fit(X_train,y_train)
y_pred2 = svc.predict(X_test)
accuracy_score(y_test,y_pred2)